# AirBnb

## New York City Airbnb Data Cleaning

Airbnb, Inc is an American company that operates an online marketplace for lodging, primarily homestays for vacation rentals, and tourism activities. Based in San Francisco, California, the platform is accessible via website and mobile app. Airbnb does not own any of the listed properties; instead, it profits by receiving commission from each booking. The company was founded in 2008. Airbnb is a shortened version of its original name, AirBedandBreakfast.com.

## About Dataset

**Context**

Since 2008, guests and hosts have used Airbnb to travel in a more unique, personalized way. As part of the Airbnb Inside initiative, this dataset describes the listing activity of homestays in New York City

**Content**

The following Airbnb activity is included in this New York dataset:

Listings, including full descriptions and average review score Reviews, including unique id for each reviewer and detailed comments Calendar, including listing id and the price and availability for that day

**Data Dictionary**

Data dictionaries are used to provide detailed information about the contents of a dataset or database, such as the names of measured variables, their data types or formats, and text descriptions. A data dictionary provides a concise guide to understanding and using the data.
https://docs.google.com/spreadsheets/d/1b_dvmyhb_kAJhUmv81rAxl4KcXn0Pymz

## Imports, Reading In Data, Preliminary Cleaning

In [1]:
# Standard Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Reading in data
df = pd.read_csv('Data/Airbnb_Open_Data.csv')
print(df.shape) # examining shape
df.head() # previewing data

(102599, 26)


/var/folders/pj/j8qg9t055bx9yt_m61ngrhyh0000gn/T/ipykernel_12433/121379091.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data/Airbnb_Open_Data.csv')


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


In [3]:
# snakecasing all columns
df.columns = [col.replace(' ','_').lower() for col in df.columns]

### Viewing Data Types

In [4]:
df.dtypes
# host_identify_verified and instant_bookable can likely become a binary column
# noticing there are two neighborhood groups
# price and service fee should be a float

id                                  int64
name                               object
host_id                             int64
host_identity_verified             object
host_name                          object
neighbourhood_group                object
neighbourhood                      object
lat                               float64
long                              float64
country                            object
country_code                       object
instant_bookable                   object
cancellation_policy                object
room_type                          object
construction_year                 float64
price                              object
service_fee                        object
minimum_nights                    float64
number_of_reviews                 float64
last_review                        object
reviews_per_month                 float64
review_rate_number                float64
calculated_host_listings_count    float64
availability_365                  

### Adjusting Data Types:
- Host Identity verified will become binary 1 represents verification, all .28% of NaN values will default to unverrified which seems reasonable given they are not confirmed to be verrified.
- instant_bookable will become binary as well, 1 represents instantly bookable, all non instant bookable including null will default to not instantly bookable.
- Price and Service Fee become floats

In [5]:
# making dummy columns
df = pd.get_dummies(df,columns=(['host_identity_verified','instant_bookable']),drop_first=True)

# df.price contains floats and strings, will make it soley a str first
df.price = df.price.astype(str) 
# removing spaces, quotations, commas, and '$'
df.price = [each.replace(' ','').replace('"','').replace(',','').strip('$') for each in df.price]
# reverting back to float
df.price = df.price.astype(float)
# df.price contains floats and strings, will make it soley a str first

df.service_fee = df.service_fee.astype(str) 
# removing spaces, quotations, commas, and '$'
df.service_fee = [each.replace(' ','').replace('"','').replace(',','').strip('$') for each in df.service_fee]
# reverting back to float
df.service_fee = df.service_fee.astype(float)

### Renaming Columns

In [6]:
df.rename(columns={'neighbourhood_group':'borough', # appropriate name
                   'host_identity_verified_verified':'verified', # simplifying
                   'instant_bookable_True':'instant_bookable'},inplace=True) # simplifying
df.borough.value_counts() # two mispelled boroughs to fix

Manhattan        43792
Brooklyn         41842
Queens           13267
Bronx             2712
Staten Island      955
brookln              1
manhatan             1
Name: borough, dtype: int64

In [7]:
df.borough.replace({"brookln": "Brooklyn", "manhatan": "Manhattan"},inplace=True)

### Viewing Null Values

In [8]:
df.isnull().sum()

id                                     0
name                                 250
host_id                                0
host_name                            406
borough                               29
neighbourhood                         16
lat                                    8
long                                   8
country                              532
country_code                         131
cancellation_policy                   76
room_type                              0
construction_year                    214
price                                247
service_fee                          273
minimum_nights                       409
number_of_reviews                    183
last_review                        15893
reviews_per_month                  15879
review_rate_number                   326
calculated_host_listings_count       319
availability_365                     448
house_rules                        52131
license                           102597
verified        

In [9]:
df.corr().price.sort_values(ascending=False)

price                             1.000000
service_fee                       0.999991
id                                0.006739
number_of_reviews                 0.005229
reviews_per_month                 0.004186
host_id                           0.003837
long                              0.003269
instant_bookable                  0.000312
calculated_host_listings_count   -0.000106
availability_365                 -0.002833
minimum_nights                   -0.003471
construction_year                -0.003753
verified                         -0.004303
review_rate_number               -0.004576
lat                              -0.005900
Name: price, dtype: float64

### First I can drop unnecessary columns
- After examining the read in error on column 25 'license', 99.9% of the column is null (102597 out of 102599 rows) so I will drop license.
- country and country_code are the same column and both depict that every listing is in the U.S. I will drop these columns because I know that every listing is in the U.S.
- last_review and reviews_per_month are redundant and contain many nulls, additionally number_of_reviews is a better indicator when viewing a listing.
- calculated_host_listings_count is similar to availability_365 that I feel comfortable removing this column
- lastly house rules contains many nulls and will be dropped.

In [10]:
df.drop(columns=['license','country','country_code','last_review',
                 'reviews_per_month','calculated_host_listings_count','house_rules'],
        inplace=True)

### Handling Null Values

In [11]:
df.shape

(102599, 19)

**First I only keep data where I am comfortable droppping nulls from specific rows**
- I didn't find a reliable way to fill these specific null values

In [12]:
df = df.loc[(df.neighbourhood.notna()) # 16 Null rows out of 102256 rows
            & (df.lat.notna()) # 8 Null rows out of 102256 rows
            & (df.cancellation_policy.notna()) # 74 Null rows out of 102256 rows
            & (df.name.notna())] # 245 Null rows out of 102256 rows

In [13]:
df.shape

(102256, 19)

**Now I can fill in null values based on the host_id**
- host_id is Airbnb's unique identifier for the host/user

In [14]:
# host name is straight forward
df['host_name'] = df.groupby('host_id')['host_name'].ffill().bfill()

# grouping by host_id will allow for construction year to be filled appropriately
df['construction_year'] = df.groupby('host_id')[['construction_year']].ffill().bfill()

# price
df.price = df.groupby(['neighbourhood','host_id'])['price'].ffill().bfill()

# service fee
df.service_fee = df.groupby('host_id')['service_fee'].ffill().bfill()

# number of reviews
df.number_of_reviews = df.groupby('host_id')['number_of_reviews'].ffill().bfill()

# review rate
df.review_rate_number = df.groupby('host_id')['review_rate_number'].ffill().bfill()

# availability
df.availability_365 = df.groupby('host_id')['availability_365'].ffill().bfill()

In [15]:
df.shape

(102256, 19)

### Filling Missing Boroughs

In [16]:
def column_summary(column):
    null = column.isnull().sum()
    rows = df.shape[0]
    null_percent = str((null/rows)*100)[:4]
    print(f"There are {null} Null rows out of {rows} rows, {null_percent}% of the column is null")
    print(f"Value Counts: \n{column.value_counts()}")

In [17]:
column_summary(df.borough)

There are 26 Null rows out of 102256 rows, 0.02% of the column is null
Value Counts: 
Manhattan        43657
Brooklyn         41686
Queens           13231
Bronx             2703
Staten Island      953
Name: borough, dtype: int64


In [18]:
df[df.borough.isnull()][['neighbourhood']].neighbourhood.unique()

array(['Washington Heights', 'Clinton Hill', 'East Village',
       'Upper East Side', 'Woodside', 'Williamsburg', 'Bushwick',
       'Prospect Heights', 'Chelsea', 'East Harlem', 'Eastchester',
       'Harlem', 'Queens Village', 'Bedford-Stuyvesant',
       'Upper West Side'], dtype=object)

In [19]:
# df.loc[(df.neighbourhood == "Upper West Side") & (df.borough)][:3]

**Can simply create a dictionary to fill in the missing boroughs based on associated neighbourhood**

In [20]:
b_dict = { # Manhattan
          'Washington Heights':'Manhattan','East Village':'Manhattan',
          'Upper East Side':'Manhattan','Chelsea':'Manhattan',
          'East Harlem':'Manhattan','Harlem':'Manhattan',
          'Chinatown':'Manhattan','Upper West Side':'Manhattan',
           # Brooklyn
          'Clinton Hill':'Brooklyn','Williamsburg':'Brooklyn',
          'Bushwick':'Brooklyn','Prospect Heights':'Brooklyn','Bedford-Stuyvesant':'Brooklyn',
           # Queens
          'Woodside':'Queens','Queens Village':'Queens',
           # Bronx
          'Eastchester':'Bronx'
         }

df.borough = df.borough.fillna(df.neighbourhood.map(b_dict)) # filling borough with dictionary

column_summary(df.borough)

There are 0 Null rows out of 102256 rows, 0.0% of the column is null
Value Counts: 
Manhattan        43669
Brooklyn         41697
Queens           13233
Bronx             2704
Staten Island      953
Name: borough, dtype: int64


### Checking cases where the borough is in the neighbourhood name
- The value counts for if the string contains a borough should be an unique instane where the name of the neighborhood contains the borough intently. 
- The sum of value_counts should equal the sum of unique instances

#### Brooklyn

In [21]:
df[df['neighbourhood'].str.contains("Brooklyn", na=False)].neighbourhood.value_counts()

Brooklyn Heights     306
Downtown Brooklyn    174
Name: neighbourhood, dtype: int64

In [22]:
df.neighbourhood.str.contains('Brooklyn').value_counts() # 308 + 175 = 483

False    101776
True        480
Name: neighbourhood, dtype: int64

#### Bronx

In [23]:
df[df['neighbourhood'].str.contains("Bronx", na=False)].neighbourhood.value_counts()

Bronxdale    47
Name: neighbourhood, dtype: int64

In [24]:
df.neighbourhood.str.contains('Bronx').value_counts()

False    102209
True         47
Name: neighbourhood, dtype: int64

#### Queens

In [25]:
df[df['neighbourhood'].str.contains("Queens", na=False)].neighbourhood.value_counts()

Queens Village    146
Name: neighbourhood, dtype: int64

In [26]:
df.neighbourhood.str.contains('Queens').value_counts()

False    102110
True        146
Name: neighbourhood, dtype: int64

#### Manhattan and Staten Island

In [27]:
df[df['neighbourhood'].str.contains("Manhattan", na=False)].neighbourhood.value_counts()

Manhattan Beach    22
Name: neighbourhood, dtype: int64

In [28]:
df.neighbourhood.str.contains('Manhattan').value_counts()

False    102234
True         22
Name: neighbourhood, dtype: int64

In [29]:
df[df['neighbourhood'].str.contains("Bay Terrace", na=False)].neighbourhood.value_counts()

Bay Terrace                   8
Bay Terrace, Staten Island    4
Name: neighbourhood, dtype: int64

In [30]:
df[df['neighbourhood'].str.contains("Chelsea", na=False)].neighbourhood.value_counts()

Chelsea                   2276
Chelsea, Staten Island       1
Name: neighbourhood, dtype: int64

In [31]:
df[df['neighbourhood'].str.contains("Staten Island", na=False)].neighbourhood.value_counts()

Bay Terrace, Staten Island    4
Chelsea, Staten Island        1
Name: neighbourhood, dtype: int64

In [32]:
# for consistency I am keeping the associated borough next to repeating neighbourhood names
df.neighbourhood.replace({"Chelsea": "Chelsea, Manhattan", 
                          "Bay Terrace": "Bay Terrace, Queens"},inplace=True)

### Minimum Nights
As such, any stay that extends for 28 days or longer is regarded as a long-term or monthly stay. Not all rental listings on Airbnb offer long-term stays since it’s the host’s job to decide on whether to make their properties available for such arrangements.
https://www.mashvisor.com/blog/airbnb-monthly-stays/

In [33]:
df.minimum_nights.describe()

count    101854.000000
mean          8.117973
std          30.271371
min       -1223.000000
25%           2.000000
50%           3.000000
75%           5.000000
max        5645.000000
Name: minimum_nights, dtype: float64

In [34]:
print(f"Shape of Dataframe by dropping column: {df.drop(columns='minimum_nights').shape}") # comparing shape

print(f"Shape of Dataframe with parameters met in accordance to airbnb rules: \
{df.loc[(df.minimum_nights >= 0) & (df.minimum_nights <=28)].shape}") # comparing shape

df.drop(columns='minimum_nights',inplace=True) # dropping column

Shape of Dataframe by dropping column: (102256, 18)
Shape of Dataframe with parameters met in accordance to airbnb rules: (87960, 19)


I will drop this column instead of keeping specific parameters for two reasons. 
- The data dictionary to this dataset has a column called "minimum_minimum_nights" which is defined as "the smallest minimum_night value from the calender (looking 365 nights in the future)." I believe there are many errors in minimum nights because data has merged or leaked in some way from minimum minimum nights
- I keep more data overall by dropping minimum nights

In [35]:
df.shape

(102256, 18)

## Original Approach

In [7]:
# examining read-in error on column 25
df.iloc[:,25]

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
102594    NaN
102595    NaN
102596    NaN
102597    NaN
102598    NaN
Name: license, Length: 102599, dtype: object

In [30]:
# Since there are only 26 columns I am creating a function to summarize what I need for cleaning if I need to
# dive into a specific column
def column_summary(column):
    null = column.isnull().sum()
    rows = df.shape[0]
    null_percent = str((null/rows)*100)[:4]
    print(f"There are {null} Null rows out of {rows} rows, {null_percent}% of the column is null")
    print(f"Value Counts: \n{column.value_counts()}")

In [9]:
column_summary(df.license)

There are 102597 Null rows out of 102599 rows, 99.9% of the column is null
Value Counts: 
41662/AL    2
Name: license, dtype: int64


I am going to drop this column due to 99% of the values being Null

In [10]:
df.drop(columns='license',inplace=True)

In [11]:
column_summary(df.host_identity_verified)

There are 289 Null rows out of 102599 rows, 0.28% of the column is null
Value Counts: 
unconfirmed    51200
verified       51110
Name: host_identity_verified, dtype: int64


Making this column binary, a 1 represents verification, all .28% of NaN values will default to unverrified which seems reasonable given they are not confirmed to be verrified.

In [12]:
df.host_identity_verified = pd.get_dummies(df.host_identity_verified,drop_first=True)

In [13]:
df.head()

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,...,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules
0,1001254,Clean & quiet apt home by the park,80014485718,0,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$966,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...
1,1002102,Skylit Midtown Castle,52335172823,1,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$142,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,0,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$620,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and..."
3,1002755,NaN,85098326012,0,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$368,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,1,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$204,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th..."


In [14]:
df.neighbourhood_group.value_counts()
# can rename this to borough to avoid confusion. Additionally there are two mispelled boroughs

Manhattan        43792
Brooklyn         41842
Queens           13267
Bronx             2712
Staten Island      955
brookln              1
manhatan             1
Name: neighbourhood_group, dtype: int64

In [15]:
df.rename(columns={'neighbourhood_group':'borough'},inplace=True)
df.borough.replace({"brookln": "Brooklyn", "manhatan": "Manhattan"},inplace=True)

### Missing Boroughs

In [16]:
column_summary(df.borough)

There are 29 Null rows out of 102599 rows, 0.02% of the column is null
Value Counts: 
Manhattan        43793
Brooklyn         41843
Queens           13267
Bronx             2712
Staten Island      955
Name: borough, dtype: int64


In [17]:
df[df.borough.isnull()][['borough','neighbourhood','lat','long']].neighbourhood.unique()

array(['Washington Heights', 'Clinton Hill', 'East Village',
       'Upper East Side', 'Woodside', 'Williamsburg', 'Bushwick',
       'Prospect Heights', 'Chelsea', 'East Harlem', 'Eastchester',
       'Harlem', 'Chinatown', 'Queens Village', 'Bedford-Stuyvesant',
       'Upper West Side'], dtype=object)

In [18]:
# df.loc[(df.neighbourhood == "Upper West Side") & (df.borough)][:3]

Can simply create a dictionary to fill in the missing boroughs based on associated neighbourhood

In [19]:
b_dict = { # Manhattan
          'Washington Heights':'Manhattan','East Village':'Manhattan',
          'Upper East Side':'Manhattan','Chelsea':'Manhattan',
          'East Harlem':'Manhattan','Harlem':'Manhattan',
          'Chinatown':'Manhattan','Upper West Side':'Manhattan',
           # Brooklyn
          'Clinton Hill':'Brooklyn','Williamsburg':'Brooklyn',
          'Bushwick':'Brooklyn','Prospect Heights':'Brooklyn','Bedford-Stuyvesant':'Brooklyn',
           # Queens
          'Woodside':'Queens','Queens Village':'Queens',
           # Bronx
          'Eastchester':'Bronx'
         }

df.borough = df.borough.fillna(df.neighbourhood.map(b_dict)) # filling borough with dictionary

column_summary(df.borough)

There are 0 Null rows out of 102599 rows, 0.0% of the column is null
Value Counts: 
Manhattan        43806
Brooklyn         41856
Queens           13269
Bronx             2713
Staten Island      955
Name: borough, dtype: int64


### Neighbourhood

In [20]:
df.neighbourhood.value_counts()

Bedford-Stuyvesant        7937
Williamsburg              7775
Harlem                    5466
Bushwick                  4982
Hell's Kitchen            3965
                          ... 
Rossville                    3
Lighthouse Hill              3
Fort Wadsworth               2
Glen Oaks                    2
Chelsea, Staten Island       1
Name: neighbourhood, Length: 224, dtype: int64

### Checking and ensuring each borough belongs in neighbourhood name
- The value counts for if the string contains a borough should be an unique instane where the name of the neighborhood contains the borough intently. 
- The sum of value_counts should equal the sum of unique instances

#### Brooklyn

In [21]:
df[df['neighbourhood'].str.contains("Brooklyn", na=False)].neighbourhood.value_counts()

Brooklyn Heights     308
Downtown Brooklyn    175
Name: neighbourhood, dtype: int64

In [22]:
df.neighbourhood.str.contains('Brooklyn').value_counts() # 308 + 175 = 483

False    102100
True        483
Name: neighbourhood, dtype: int64

#### Bronx

In [23]:
df[df['neighbourhood'].str.contains("Bronx", na=False)].neighbourhood.value_counts()

Bronxdale    48
Name: neighbourhood, dtype: int64

In [24]:
df.neighbourhood.str.contains('Bronx').value_counts()

False    102535
True         48
Name: neighbourhood, dtype: int64

#### Queens

In [25]:
df[df['neighbourhood'].str.contains("Queens", na=False)].neighbourhood.value_counts()

Queens Village    147
Name: neighbourhood, dtype: int64

In [26]:
df.neighbourhood.str.contains('Queens').value_counts()

False    102436
True        147
Name: neighbourhood, dtype: int64

#### Manhattan and Staten Island

In [27]:
df[df['neighbourhood'].str.contains("Manhattan", na=False)].neighbourhood.value_counts()

Manhattan Beach    22
Name: neighbourhood, dtype: int64

In [28]:
df.neighbourhood.str.contains('Manhattan').value_counts()

False    102561
True         22
Name: neighbourhood, dtype: int64

In [29]:
df[df['neighbourhood'].str.contains("Bay Terrace", na=False)].neighbourhood.value_counts()

Bay Terrace                   8
Bay Terrace, Staten Island    4
Name: neighbourhood, dtype: int64

In [30]:
df[df['neighbourhood'].str.contains("Chelsea", na=False)].neighbourhood.value_counts()

Chelsea                   2285
Chelsea, Staten Island       1
Name: neighbourhood, dtype: int64

In [31]:
df[df['neighbourhood'].str.contains("Staten Island", na=False)].neighbourhood.value_counts()

Bay Terrace, Staten Island    4
Chelsea, Staten Island        1
Name: neighbourhood, dtype: int64

In [32]:
# for consistency I am keeping the associated borough next to repeating neighbourhood names
df.neighbourhood.replace({"Chelsea": "Chelsea, Manhattan", 
                          "Bay Terrace": "Bay Terrace, Queens"},inplace=True)

### Neighbourhoods

In [33]:
column_summary(df.neighbourhood)

There are 16 Null rows out of 102599 rows, 0.01% of the column is null
Value Counts: 
Bedford-Stuyvesant        7937
Williamsburg              7775
Harlem                    5466
Bushwick                  4982
Hell's Kitchen            3965
                          ... 
Rossville                    3
Lighthouse Hill              3
Fort Wadsworth               2
Glen Oaks                    2
Chelsea, Staten Island       1
Name: neighbourhood, Length: 224, dtype: int64


I am comfortable dropping this data since the total amount of data dropped will not be significant as opposed to pinpointing the neighborhood based on latitude and longitude.

In [34]:
df.shape

(102599, 25)

In [35]:
df.neighbourhood.describe()

count                 102583
unique                   224
top       Bedford-Stuyvesant
freq                    7937
Name: neighbourhood, dtype: object

In [36]:
df = df[df.neighbourhood.notna()]

In [37]:
df.neighbourhood.describe()

count                 102583
unique                   224
top       Bedford-Stuyvesant
freq                    7937
Name: neighbourhood, dtype: object

In [38]:
df.shape

(102583, 25)

### Host Name

In [39]:
column_summary(df.host_name)

There are 406 Null rows out of 102583 rows, 0.39% of the column is null
Value Counts: 
Michael         881
David           764
John            581
Alex            546
Sonder (NYC)    516
               ... 
Sonskeshana       1
Soph              1
Anshu             1
Hayet             1
Shuhel            1
Name: host_name, Length: 13190, dtype: int64


In [40]:
df.host_name.describe()

count      102177
unique      13190
top       Michael
freq          881
Name: host_name, dtype: object

In [41]:
# can simply fill the missing host name by associated id since there are no missing id's
df['host_name'] = df.groupby('host_id')['host_name'].ffill().bfill()

In [42]:
df.host_name.describe()

count      102583
unique      13190
top       Michael
freq          882
Name: host_name, dtype: object

In [43]:
column_summary(df.host_name)

There are 0 Null rows out of 102583 rows, 0.0% of the column is null
Value Counts: 
Michael            882
David              772
John               586
Alex               546
Sonder (NYC)       517
                  ... 
Zhanhong             1
Kayser               1
Nachi                1
Rabi & Soukaina      1
Egon                 1
Name: host_name, Length: 13190, dtype: int64


### Latitude and Longitude

In [44]:
df[df.lat.isnull()]

,id,name,host_id,host_identity_verified,host_name,borough,neighbourhood,lat,long,country,...,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules
779,1431578,"Large, furnished room in a 2 bedroom!",20368956893,0,Gibson,Brooklyn,Crown Heights,NaN,NaN,United States,...,$539,$108,1.0,1.0,3/18/2017,0.04,2.0,1.0,41.0,- Weekly and monthly prices are much lower - P...
785,1434892,Authentic NY Charming Artist Loft,66486085219,0,Bailey,Brooklyn,Greenpoint,NaN,NaN,United States,...,"$1,058",$212,5.0,14.0,6/19/2019,0.16,5.0,1.0,226.0,We live and let live - hoping that you'd be re...
799,1442624,Huge room with private balcony,69386945815,1,Hunt,Manhattan,East Village,NaN,NaN,United States,...,$506,$101,6.0,1.0,5/6/2013,0.01,1.0,1.0,240.0,Expect respect for the family and the space--t...
814,1450908,Decorators 5-Star Flat West Village,33280739304,1,Watson,Manhattan,West Village,NaN,NaN,United States,...,$381,$76,20.0,157.0,8/11/2016,1.71,4.0,1.0,61.0,"Please keep it clean, thats all we really ask ..."
843,1466925,Nice Private Room Beauty in Queens,15305733205,1,Roberts,Queens,Elmhurst,NaN,NaN,United States,...,$224,$45,1.0,63.0,5/18/2019,0.89,3.0,2.0,70.0,NaN
885,1490122,Cute Room in Historic Loft!,42267829819,0,Jones,Brooklyn,Greenpoint,NaN,NaN,United States,...,$524,$105,14.0,22.0,5/2/2019,0.25,1.0,1.0,266.0,"Pets are cool (just clean up after them!), smo..."
926,1512766,21 day Chelsea Apartment rental,10876728736,0,Owens,Manhattan,Flatiron District,NaN,NaN,United States,...,$623,$125,21.0,0.0,NaN,NaN,2.0,1.0,104.0,NaN
986,1545904,New York City for All Seasons!,26437872336,0,Douglas,Manhattan,Upper West Side,NaN,NaN,United States,...,$413,$83,1.0,25.0,6/22/2013,0.28,2.0,1.0,259.0,No Smoking No Pets


In [45]:
#df.groupby(['borough','neighbourhood'])['lat','long'].ffill().bfill()

In [46]:
# The same 8 rows for latitude and longitude are null
# Given the small amount I will just drop these rows
df = df[df.lat.notna()]

In [47]:
df.shape

(102575, 25)

### Country and Country Code

In [48]:
# duplicate columns, dropping country_code, renaming all United States to US
df['country'] = ['US']*len(df)
df.drop(columns={'country_code'},inplace=True)
df.drop(columns='minimum_nights',inplace=True)

### Instant Bookable

In [49]:
# making this a dummy column, all null values will default to not instantly bookable
# only keeping instance as instantly bookable if it is known to be true
df.instant_bookable = pd.get_dummies(df.instant_bookable,drop_first=True)

In [50]:
df.isnull().sum()

id                                    0
name                                250
host_id                               0
host_identity_verified                0
host_name                             0
borough                               0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
instant_bookable                      0
cancellation_policy                  74
room_type                             0
construction_year                   213
price                               246
service_fee                         273
minimum_nights                      408
number_of_reviews                   183
last_review                       15892
reviews_per_month                 15878
review_rate_number                  326
calculated_host_listings_count      319
availability_365                    447
house_rules                       52123
dtype: int64

### Cancellation Policy

In [51]:
# dropping null values, there are 74 missing values and I don't see a reliable way to fill these missing values
df = df[df.cancellation_policy.notna()]

In [52]:
df[df.construction_year.isnull()]

,id,name,host_id,host_identity_verified,host_name,borough,neighbourhood,lat,long,country,...,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules
118,1066507,NaN,87894881413,0,David,Manhattan,Upper West Side,40.77823,-73.97637,US,...,"$1,154",$231,6.0,27.0,4/27/2019,0.27,NaN,NaN,237.0,"This is our family home, not a rental that we ..."
119,1067060,Great Large 1 BR apt in East Village!,27506185339,0,Adelaide,Manhattan,East Village,40.72555,-73.97965,US,...,$737,$147,1.0,115.0,6/7/2019,1.05,NaN,NaN,242.0,Please pick up after yourself if you use the k...
120,1067612,Eveland Private Bed & Living Room w/ Own Entrance,25474679480,0,Fenton,Brooklyn,South Slope,40.66831,-73.98604,US,...,$456,$91,2.0,354.0,5/20/2019,3.20,NaN,NaN,227.0,- There is no dedicated parking spot for this ...
191,1106825,LUX APT IN TIMES SQUARE NEW BUILDING,93725364475,0,Aiden,Manhattan,Hell's Kitchen,40.76307,-73.99665,US,...,$539,$108,NaN,41.0,6/26/2025,0.38,2.0,1.0,NaN,"You will be sharing a bathroom, so you must be..."
192,1107378,Designer 1 BR Duplex w/ Terrace- Spectacular V...,11711269275,1,Antony,Manhattan,Lower East Side,40.71882,-73.98852,US,...,$232,$46,NaN,109.0,6/15/2019,1.04,2.0,1.0,NaN,"You're welcome to use our kitchen, we just ask..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102052,57364656,"☀️Private, cozy & quiet room in Inwood Manhatt...",74740790774,0,Zhoe,Manhattan,Inwood,40.86857,-73.91577,US,...,"$1,183",NaN,4.0,95.0,7/1/2019,3.65,2.0,1.0,323.0,This place is about enjoying good times and th...
102053,57365208,Cozy bright room near Prospect Park,77326652202,0,Mariam,Brooklyn,Flatbush,40.64945,-73.96108,US,...,$696,NaN,7.0,12.0,3/27/2019,0.44,5.0,1.0,0.0,Shoes off Clean After yourself Turn Lights and...
102054,57365760,Private Bedroom with Amazing Rooftop View,45936254757,1,Trey,Brooklyn,Bushwick,40.69872,-73.92718,US,...,$909,NaN,1.0,19.0,8/31/2017,0.72,3.0,2.0,0.0,#NAME?
102055,57366313,Pretty Brooklyn One-Bedroom for 2 to 4 people,23801060917,1,Michael,Brooklyn,Bedford-Stuyvesant,40.67810,-73.90822,US,...,$387,NaN,2.0,50.0,6/26/2019,3.12,4.0,2.0,235.0,* Check out: 10am * We made an effort to keep ...


In [53]:
df.shape

(102501, 24)

In [54]:
df.construction_year.describe()

count    102297.000000
mean       2012.487473
std           5.765232
min        2003.000000
25%        2007.000000
50%        2012.000000
75%        2017.000000
max        2022.000000
Name: construction_year, dtype: float64

In [55]:
df['construction_year'] = df.groupby('host_id')[['construction_year']].ffill().bfill()
# again I will use the host_id to fill in the null values for the construction year

In [56]:
df.construction_year.describe()
# no significant changes, only one year added to the 25th and 75th percentile

count    102501.000000
mean       2012.491195
std           5.765175
min        2003.000000
25%        2008.000000
50%        2012.000000
75%        2018.000000
max        2022.000000
Name: construction_year, dtype: float64

In [57]:
df.isnull().sum()

id                                    0
name                                245
host_id                               0
host_identity_verified                0
host_name                             0
borough                               0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
instant_bookable                      0
cancellation_policy                   0
room_type                             0
construction_year                     0
price                               242
service_fee                         273
minimum_nights                      408
number_of_reviews                   183
last_review                       15883
reviews_per_month                 15872
review_rate_number                  326
calculated_host_listings_count      319
availability_365                    440
house_rules                       52090
dtype: int64

In [58]:
df.price

0           $966 
1           $142 
2           $620 
3           $368 
4           $204 
           ...   
102594      $844 
102595      $837 
102596      $988 
102597      $546 
102598    $1,032 
Name: price, Length: 102501, dtype: object

In [68]:
# df.price contains floats and strings, will make it soley a str first
df.price = df.price.astype(str) 
# removing spaces, quotations, commas, and '$'
df.price = [each.replace(' ','').replace('"','').replace(',','').strip('$') for each in df.price]
# reverting back to float
df.price = df.price.astype(float)

In [74]:
df.price.describe()

count    102259.000000
mean        625.274959
std         331.694504
min          50.000000
25%         339.000000
50%         624.000000
75%         913.000000
max        1200.000000
Name: price, dtype: float64

In [82]:
# filling null values based on neighbourhood associated and host_id
df.price = df.groupby(['neighbourhood','host_id'])['price'].ffill().bfill()

In [83]:
df.price.describe()

count    102501.000000
mean        625.167989
std         331.776996
min          50.000000
25%         339.000000
50%         624.000000
75%         913.000000
max        1200.000000
Name: price, dtype: float64

In [84]:
df.isnull().sum()

id                                    0
name                                245
host_id                               0
host_identity_verified                0
host_name                             0
borough                               0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
instant_bookable                      0
cancellation_policy                   0
room_type                             0
construction_year                     0
price                                 0
service_fee                         273
minimum_nights                      408
number_of_reviews                   183
last_review                       15883
reviews_per_month                 15872
review_rate_number                  326
calculated_host_listings_count      319
availability_365                    440
house_rules                       52090
dtype: int64

In [85]:
df.service_fee

0         $193 
1          $28 
2         $124 
3          $74 
4          $41 
          ...  
102594    $169 
102595    $167 
102596    $198 
102597    $109 
102598    $206 
Name: service_fee, Length: 102501, dtype: object

In [86]:
# df.price contains floats and strings, will make it soley a str first
df.service_fee = df.service_fee.astype(str) 
# removing spaces, quotations, commas, and '$'
df.service_fee = [each.replace(' ','').replace('"','').replace(',','').strip('$') for each in df.service_fee]
# reverting back to float
df.service_fee = df.service_fee.astype(float)

In [89]:
df.service_fee.describe()

count    102228.000000
mean        125.024807
std          66.330690
min          10.000000
25%          68.000000
50%         125.000000
75%         183.000000
max         240.000000
Name: service_fee, dtype: float64

In [96]:
df.service_fee = df.groupby('host_id')['service_fee'].ffill().bfill()

In [98]:
df.service_fee.describe()

count    102501.000000
mean        125.007990
std          66.327009
min          10.000000
25%          68.000000
50%         125.000000
75%         182.000000
max         240.000000
Name: service_fee, dtype: float64

In [97]:
df.isnull().sum()

id                                    0
name                                245
host_id                               0
host_identity_verified                0
host_name                             0
borough                               0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
instant_bookable                      0
cancellation_policy                   0
room_type                             0
construction_year                     0
price                                 0
service_fee                           0
minimum_nights                      408
number_of_reviews                   183
last_review                       15883
reviews_per_month                 15872
review_rate_number                  326
calculated_host_listings_count      319
availability_365                    440
house_rules                       52090
dtype: int64

### Minimum Nights
As such, any stay that extends for 28 days or longer is regarded as a long-term or monthly stay. Not all rental listings on Airbnb offer long-term stays since it’s the host’s job to decide on whether to make their properties available for such arrangements.
https://www.mashvisor.com/blog/airbnb-monthly-stays/

In [33]:
df.minimum_nights.describe()

count    101854.000000
mean          8.117973
std          30.271371
min       -1223.000000
25%           2.000000
50%           3.000000
75%           5.000000
max        5645.000000
Name: minimum_nights, dtype: float64

In [34]:
print(f"Shape of Dataframe by dropping column: {df.drop(columns='minimum_nights').shape}") # comparing shape

print(f"Shape of Dataframe with parameters met in accordance to airbnb rules: \
{df.loc[(df.minimum_nights >= 0) & (df.minimum_nights <=28)].shape}") # comparing shape

df.drop(columns='minimum_nights',inplace=True) # dropping column

Shape of Dataframe by dropping column: (102256, 18)
Shape of Dataframe with parameters met in accordance to airbnb rules: (87960, 19)


I will drop this column instead of keeping specific parameters for two reasons. 
- The data dictionary to this dataset has a column called "minimum_minimum_nights" which is defined as "the smallest minimum_night value from the calender (looking 365 nights in the future)." I believe there are many errors in minimum nights because data has merged or leaked in some way from minimum minimum nights
- I keep more data overall by dropping minimum nights

In [125]:
df.isnull().sum()

id                                    0
name                                245
host_id                               0
host_identity_verified                0
host_name                             0
borough                               0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
instant_bookable                      0
cancellation_policy                   0
room_type                             0
construction_year                     0
price                                 0
service_fee                           0
number_of_reviews                   183
last_review                       15883
reviews_per_month                 15872
review_rate_number                  326
calculated_host_listings_count      319
availability_365                    440
house_rules                       52090
dtype: int64

In [133]:
df.number_of_reviews.describe()

count    102318.000000
mean         27.456019
std          49.463757
min           0.000000
25%           1.000000
50%           7.000000
75%          30.000000
max        1024.000000
Name: number_of_reviews, dtype: float64

In [134]:
df.number_of_reviews = df.groupby('host_id')['number_of_reviews'].ffill().bfill()

In [135]:
df.number_of_reviews.describe()

count    102501.000000
mean         27.442659
std          49.479439
min           0.000000
25%           1.000000
50%           7.000000
75%          30.000000
max        1024.000000
Name: number_of_reviews, dtype: float64

In [139]:
# excessive nulls and avoiding multicolinearity
df.drop(columns=['last_review','reviews_per_month'],inplace=True)

In [140]:
df.isnull().sum()

id                                    0
name                                245
host_id                               0
host_identity_verified                0
host_name                             0
borough                               0
neighbourhood                         0
lat                                   0
long                                  0
country                               0
instant_bookable                      0
cancellation_policy                   0
room_type                             0
construction_year                     0
price                                 0
service_fee                           0
number_of_reviews                     0
review_rate_number                  326
calculated_host_listings_count      319
availability_365                    440
house_rules                       52090
dtype: int64

In [141]:
df.review_rate_number.describe()

count    102175.000000
mean          3.279168
std           1.284576
min           1.000000
25%           2.000000
50%           3.000000
75%           4.000000
max           5.000000
Name: review_rate_number, dtype: float64

In [148]:
df.review_rate_number = df.groupby('host_id')['review_rate_number'].ffill().bfill()

In [150]:
# keeping availability 365 as oppossed to calculated host listings count
# house rules has an excess amount of nulls 
df.drop(columns=['calculated_host_listings_count','house_rules'],inplace=True)

In [151]:
df.isnull().sum()

id                          0
name                      245
host_id                     0
host_identity_verified      0
host_name                   0
borough                     0
neighbourhood               0
lat                         0
long                        0
country                     0
instant_bookable            0
cancellation_policy         0
room_type                   0
construction_year           0
price                       0
service_fee                 0
number_of_reviews           0
review_rate_number          0
availability_365          440
dtype: int64

In [158]:
df = df[df.name.notna()]

In [162]:
df.availability_365.describe()

count    101833.000000
mean        141.118645
std         135.426324
min         -10.000000
25%           3.000000
50%          96.000000
75%         269.000000
max        3677.000000
Name: availability_365, dtype: float64

In [163]:
df.availability_365 = df.groupby('host_id')['availability_365'].ffill().bfill()

/var/folders/pj/j8qg9t055bx9yt_m61ngrhyh0000gn/T/ipykernel_11787/485533101.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.availability_365 = df.groupby('host_id')['availability_365'].ffill().bfill()
